In [43]:
import requests
import json
from tqdm import tqdm

In [2]:
import pandas as pd

val_df = pd.read_csv("val_df")
val_df.head()

,Unnamed: 0,instruction,input,output
0,3851,Ты - юридический консультант. Тебе даётся след...,Могут ли привлечь меня к ответственности как д...,"Нет, если вы не выполняете организационно-расп..."
1,1766,Ты - юридический консультант. Тебе даётся след...,"Обязан ли сотрудник ГИБДД, составляющий проток...","Да, обязан. Согласно ст. 25.1 КоАП РФ и ст. 51..."
2,6333,Ты - юридический консультант. Тебе даётся след...,"Могут ли аннулировать вид на жительство, если ...","Да, могут. Согласно подп. 7 п. 1 ст. 9 Федерал..."
3,8795,Ты - юридический консультант. Тебе даётся след...,Могу ли я взыскать неустойку за просрочку устр...,"Да, но с учетом ограничений. По ч. 8 ст. 7 214..."
4,2933,Ты - юридический консультант. Тебе даётся след...,"Как обжаловать постановление, если доказательс...",Жалоба подается в вышестоящий суд (ст. 30.1 Ко...


In [18]:
val_df.instruction[0]

'Ты - юридический консультант. Тебе даётся следующий контекст: Врач-невролог ФИО1 был привлечен к административной ответственности по ч. 1 ст. 5.61 КоАП РФ за оскорбление коллеги в рабочем помещении. Мировой судья признал его виновным как должностное лицо и назначил штраф в размере 30 000 рублей. ФИО1 подал жалобу, оспаривая квалификацию его как должностного лица, поскольку он не обладает организационно-распорядительными или административно-хозяйственными функциями, а дело было возбуждено в отношении него как физического лица. Исходя из этого, ответь на вопрос: '

In [19]:
for instruct, context, answer in zip(val_df['instruction'][:3], val_df["input"][:3],val_df["output"][:3]):
    print(instruct, context, answer, end="\n", sep="\n")

Ты - юридический консультант. Тебе даётся следующий контекст: Врач-невролог ФИО1 был привлечен к административной ответственности по ч. 1 ст. 5.61 КоАП РФ за оскорбление коллеги в рабочем помещении. Мировой судья признал его виновным как должностное лицо и назначил штраф в размере 30 000 рублей. ФИО1 подал жалобу, оспаривая квалификацию его как должностного лица, поскольку он не обладает организационно-распорядительными или административно-хозяйственными функциями, а дело было возбуждено в отношении него как физического лица. Исходя из этого, ответь на вопрос: 
Могут ли привлечь меня к ответственности как должностное лицо, если я обычный врач без управленческих функций?
Нет, если вы не выполняете организационно-распорядительные или административно-хозяйственные функции (примечание к ст. 2.4 КоАП РФ). Для привлечения как должностного лица необходимо, чтобы правонарушение было связано с исполнением служебных обязанностей, чего в вашем случае нет.
Ты - юридический консультант. Тебе даётся

In [17]:
"Нет, если вы не занимаете должности с организационно-распорядительными или административно-хозяйственными функциями (ст. 2.4 КоАП РФ). Квалификация как должностного лица возможна только при наличии таких полномочий."

'Нет, если вы не занимаете должности с организационно-распорядительными или административно-хозяйственными функциями (ст. 2.4 КоАП РФ). Квалификация как должностного лица возможна только при наличии таких полномочий.'

In [30]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS, Chroma
import numpy as np
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from sentence_transformers import CrossEncoder
from typing import List, Dict
from langchain.docstore.document import Document
import torch

In [25]:
embedder_labse = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru"
)

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/516M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/521k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

In [28]:
vector_store_labse = FAISS.load_local("faiss_legal_index_labse_2", embedder_labse, allow_dangerous_deserialization=True)

In [29]:
bm25_retriever_labse = BM25Retriever.from_documents(list(vector_store_labse.docstore._dict.values()), metadata_keys=["document", "part"])

In [31]:
class HierarchicalRetriever:
    def __init__(self, vector_db, bm25_retriever, reranker):
        self.faiss_k = 5
        self.bm25_k = 5
        self.weights = [0.4, 0.6]
        self.vector_db = vector_db
        self.bm25_retriever = bm25_retriever
        self.bm25_retriever.k = self.bm25_k
        self.faiss_retriever = vector_db.as_retriever(search_kwargs={"k":self.faiss_k})
        self.ensemble_retriever = EnsembleRetriever(
            retrievers=[self.bm25_retriever, self.faiss_retriever],
            weights=self.weights,
            k=self.bm25_k
        )
        self.reranker = reranker

    def ensemble_retrieve(
        self,
        query: str,
    ) -> List[Document]:

        return self.ensemble_retriever.invoke(query)

    def faiss_retrieve(
        self,
        query: str,
    ) -> List[Document]:

        return self.faiss_retriever.invoke(query)

    def bm25_retrieve(
            self,
            query: str,
    ) -> List[Document]:

        return self.bm25_retriever.invoke(query)

    def retrieve(
        self,
        query: str,
        document_filter: str = None,
        part_keywords: List[str] = None
    ) -> List[Document]:


        docs = self.ensemble_retrieve(query)
        rerank_docs = self.rerank_documents(query, docs, top_k=5)

        # Уровень 1: Фильтрация по документу
        if document_filter:
            docs = self.vector_db.similarity_search(
                query,
                filter=lambda doc: doc.metadata["document"] == document_filter
            )
        else:
            docs = self.vector_db.similarity_search(query)

        # Уровень 2: Фильтрация по разделу
        if part_keywords:
            filtered = []
            for doc in docs:
                if any(kw in doc.metadata["part"] for kw in part_keywords):
                    filtered.append(doc)
            docs = filtered

        return rerank_docs

    def rerank_documents(
            self,
            query: str,
            docs: list[Document],
            top_k: int = 5
        ) -> list[Document]:

        pairs = [(query, doc.page_content) for doc in docs]

        scores = self.reranker.predict(pairs, batch_size=32)

        scored_docs = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)

        return [doc for doc, score in scored_docs[:top_k]]

In [32]:
from sentence_transformers import CrossEncoder

reranker = CrossEncoder("cross-encoder/qnli-distilroberta-base")

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [33]:
custom_retriever_labse = HierarchicalRetriever(vector_store_labse, bm25_retriever_labse, reranker)

In [35]:
custom_retriever_labse.retrieve("Нет, если вы не занимаете должности с организационно-распорядительными или административно-хозяйственными функциями (ст. 2.4 КоАП РФ). Квалификация как должностного лица возможна только при наличии таких полномочий.")

[Document(id='b006998d-00e2-43c5-abe7-3dfcf8f4db8a', metadata={'document': 'КоАП', 'part': 'Глава 27. Применение мер обеспечения производства по делам об административных правонарушениях Статья 27.19. Помещение в специальные учреждения иностранных граждан или лиц без гражданства, подлежащих административному выдворению за пределы Российской Федерации'}, page_content='м, в случае, если судья признает обязательным присутствие в судебном заседании указанных лиц.3.10. Неявка в судебное заседание надлежащим образом извещенного уполномоченного должностного лица органа внутренних дел или органа исполнительной власти в области обеспечения безопасности, подавшего ходатайство о продлении срока содержания иностранного гражданина или лица без гражданства в специальном учреждении, не препятствует рассмотрению и разрешению такого ходатайства при наличии документов, подтве'),
 Document(id='0cb42d1e-e303-4421-8b23-d81b38c780a3', metadata={'document': 'ГК_merged', 'part': '§ 1. Основные положения Стать

In [ ]:
'м, в случае, если судья признает обязательным присутствие в судебном заседании указанных лиц.3.10. Неявка в судебное заседание надлежащим образом извещенного уполномоченного должностного лица органа внутренних дел или органа исполнительной власти в области обеспечения безопасности, подавшего ходатайство о продлении срока содержания иностранного гражданина или лица без гражданства в специальном учреждении, не препятствует рассмотрению и разрешению такого ходатайства при наличии документов, подтве'
'ение вопросов, не входящих в компетенцию ее высшего органа и созданного в соответствии с пунктом 4 настоящей статьи коллегиального органа управления.4. Наряду с исполнительными органами, указанными в пункте 3 настоящей статьи, в корпорации может быть образован в случаях, предусмотренных настоящим Кодексом, другим законом или уставом корпорации, коллегиальный орган управления (наблюдательный или иной совет), контролирующий деятельность исполнительных органов корпорации и выполняющий иные функции,'
'й Федерации;отсутствие соответствующего документа об образовании и (или) о квалификации, если выполнение работы требует специальных знаний в соответствии с федеральным законом или иным нормативным правовым актом;заключение трудового договора в нарушение постановления судьи, органа, должностного лица, уполномоченных рассматривать дела об административных правонарушениях, о дисквалификации или ином административном наказании, исключающем возможность исполнения работником обязанностей по трудовому'
'ее двух рабочих дней, следующих за днем выявления таких фактов, в письменной форме уведомить об этом Банк России (с указанием соответствующих фактов);2) не позднее одного месяца со дня выявления таких фактов в порядке, предусмотренном трудовым законодательством, освободить указанное лицо от занимаемой должности (освободить от временного исполнения обязанностей по этой должности);3) уведомить Банк России об освобождении указанного лица от занимаемой должности (освобождении от временного исполнени'
'ющее функции (лица, осуществляющие функции) единоличного исполнительного органа указанных юридических лиц, должны соответствовать следующим требованиям:отсутствие неснятой или непогашенной судимости за совершение преступления;отсутствие факта применения в отношении лица административного наказания в виде дисквалификации в соответствии с Кодексом Российской Федерации об административных правонарушениях (законодательным актом иностранного государства, если им предусмотрено такое наказание), если н'

In [ ]:
def deepseek_api_call(user_input: str, api_key: str, model: str = "deepseek-chat") -> str:
    
    # URL API DeepSeek (уточните актуальный эндпоинт в документации)
    api_url = "https://api.deepseek.com/v1/chat/completions"
    
    # Заголовки запроса
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }
    
    # Тело запроса
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": """Ты выполняешь роль эксперта, которому поручено оценить соответствие 
                                             сгенерированного искусственным интеллектом ответа данному контексту 
                                             и эталонному ответу."""},
            {"role": "user", "content": user_input}
        ],
        "temperature": 0.0,  
    }
    
    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()  # Проверка на ошибки HTTP
        
        # Парсинг ответа
        response_data = response.json()
        return response_data["choices"][0]["message"]["content"]
    
    except requests.exceptions.RequestException as e:
        return f"Ошибка API: {e}"
    except (KeyError, IndexError) as e:
        return f"Ошибка парсинга ответа: {e}"

In [ ]:
#lora_geval_res = []
base_geval_res = []
#with open('lora_geval_backup.json', 'w', encoding='utf-8') as f:
#    json.dump(lora_geval_res, f, ensure_ascii=False)
with open('base_geval_backup.json', 'w', encoding='utf-8') as f:
    json.dump(base_geval_res, f, ensure_ascii=False)

In [6]:
import pandas as pd
from tqdm import tqdm

In [12]:
contexts.columns

Index(['Unnamed: 0', 'instruction', 'input', 'output', 'model_answer',
       'labse_faiss_top0', 'labse_faiss_top1', 'labse_faiss_top2',
       'labse_faiss_top3', 'labse_faiss_top4', 'labse_bm25_top0',
       'labse_bm25_top1', 'labse_bm25_top2', 'labse_bm25_top3',
       'labse_bm25_top4', 'labse_ensemble_top0', 'labse_ensemble_top1',
       'labse_ensemble_top2', 'labse_ensemble_top3', 'labse_ensemble_top4',
       'labse_custom_top0', 'labse_custom_top1', 'labse_custom_top2',
       'labse_custom_top3', 'labse_custom_top4', 'mini_faiss_top0',
       'mini_faiss_top1', 'mini_faiss_top2', 'mini_faiss_top3',
       'mini_faiss_top4', 'mini_bm25_top0', 'mini_bm25_top1', 'mini_bm25_top2',
       'mini_bm25_top3', 'mini_bm25_top4', 'mini_ensemble_top0',
       'mini_ensemble_top1', 'mini_ensemble_top2', 'mini_ensemble_top3',
       'mini_ensemble_top4', 'mini_custom_top0', 'mini_custom_top1',
       'mini_custom_top2', 'mini_custom_top3', 'mini_custom_top4',
       'multi_faiss_top0', '

In [4]:
contexts = pd.read_csv("contexts", index_col=0)
contexts.head()

,Unnamed: 0,instruction,input,output,model_answer,labse_faiss_top0,labse_faiss_top1,labse_faiss_top2,labse_faiss_top3,labse_faiss_top4,...,multi_ensemble_top0,multi_ensemble_top1,multi_ensemble_top2,multi_ensemble_top3,multi_ensemble_top4,multi_custom_top0,multi_custom_top1,multi_custom_top2,multi_custom_top3,multi_custom_top4
0,2588,Ты - юридический консультант. Тебе даётся след...,Обязан ли инспектор проводить фотофиксацию мес...,"Нет, фотофиксация не является обязательной по ...","Нет, фотофиксация не обязательна, если иные до...",Глава 26. Предмет доказывания. Доказательства....,Глава 6. Доказательства и доказывание Статья 6...,Глава 6. Доказательства и доказывание Статья 6...,"а Российской Федерации, совершенных с использо...",Глава 26. Предмет доказывания. Доказательства....,...,"и фото- и киносъемки, иных установленных спосо...","фото- и киносъемки, иных установленных способо...","анными в статьях 27.2, 27.3 настоящего Кодекса...","владении которого находятся товары, транспортн...",присутствии двух понятых либо с применением ви...,"их доказательств перед другими и определять, к...",е документы не должны содержать ссылки на дока...,"владении которого находятся товары, транспортн...","анными в статьях 27.2, 27.3 настоящего Кодекса...","ые действия (бездействие), суд может истребова..."
1,6365,Ты - юридический консультант. Тебе даётся след...,Могут ли отменить постановление инспектора ГИБ...,"Да, могут. Согласно ст. 29.10 КоАП РФ, постано...","Да, согласно ч. 1 ст. 30.7 КоАП РФ, суд вправе...",Глава 30. Пересмотр постановлений и решений по...,"ение норм процессуального права, если оно прив...","рации, суда общей юрисдикции о признании недей...",суда первой или апелляционной инстанции и прин...,"суда и заявление о его отмене, подлежит приост...",...,ыть указано только на признание иска и приняти...,"суда и заявление о его отмене, подлежит приост...",дерации определения об отмене этого решения;2)...,о суда в случае вынесения иным судом Российско...,е о выдаче исполнительного листа на принудител...,"суда и заявление о его отмене, подлежит приост...",ыть указано только на признание иска и приняти...,о суда в случае вынесения иным судом Российско...,"отменить постановления судов первой, апелляцио...",ях могут быть устранены без возвращения дела н...
2,10807,Ты - юридический консультант. Тебе даётся след...,"Можно ли избежать штрафа, если оскорбление был...","Нет, провокация не исключает ответственность, ...","Нет, ст. 5.61 КоАП РФ не предусматривает отягч...",") обстоятельства, установленные при рассмотрен...",Глава 28. Возбуждение дела об административном...,лежащим образом извещенного прокурора не являе...,суда первой или апелляционной инстанции и прин...,"енить судебный акт суда первой, апелляционной ...",...,"рассмотрению судом, в который оно направлено. ...","ящей статьи, устанавливается судом.7. Применен...",ыть указано только на признание иска и приняти...,повлечь за собой наложение судебного штрафа в ...,"уд, давать заведомо ложные показания, отказыва...","ящей статьи, устанавливается судом.7. Применен...",повлечь за собой наложение судебного штрафа в ...,"уд, давать заведомо ложные показания, отказыва...",Глава 6. Доказательства и доказывание Статья 6...,Глава 6. Доказательства и доказывание Статья 8...
3,1120,Ты - юридический консультант. Тебе даётся след...,"На какую неустойку я могу рассчитывать, если с...","По п. 21 ст. 12 ФЗ «Об ОСАГО», за каждый день ...","По п. 21 ст. 12 Закона об ОСАГО, за каждый ден...",выплаты или срока выдачи потерпевшему направле...,ветствии с пунктом 4 статьи 10 настоящего Феде...,такому лицу путем перечисления суммы компенсац...,нь просрочки в размере одного процента от опре...,твии с законодательством о несостоятельности (...,...,выплаты или срока выдачи потерпевшему направле...,астоящим Федеральным законом суммы страхового ...,нь просрочки в размере одного процента от опре...,такому лицу путем перечисления суммы компенсац...,им пунктом неустойка (пеня) или сумма финансов...,"ельской дея

In [8]:
for i in tqdm(range(len(contexts))):
    print(contexts['output'][i])
    break

  0%|          | 0/120 [00:00<?, ?it/s]

Нет, фотофиксация не является обязательной по КоАП РФ. Согласно ст. 26.11 КоАП РФ, доказательства оцениваются в совокупности. Если другие материалы (схема, объяснения) достаточны для установления обстоятельств, отсутствие фото не повлияет на законность постановления.


In [59]:
n=0
instruction = contexts['instruction'][n]
question = contexts['input'][n]
gold_answer = contexts['output'][n]
list_contexts = [contexts[f'multi_ensemble_top{i}'][n] for i in range(5)]
print(*list_contexts, sep='\n')

и фото- и киносъемки, иных установленных способов фиксации вещественных доказательств. Материалы, полученные при осуществлении осмотра с применением фото- и киносъемки, иных установленных способов фиксации вещественных доказательств, прилагаются к соответствующему протоколу.6. Протокол об осмотре принадлежащих юридическому лицу или индивидуальному предпринимателю помещений, территорий и находящихся там вещей и документов подписывается должностным лицом, его составившим, законным представителем ю
фото- и киносъемки, иных установленных способов фиксации вещественных доказательств. Материалы, полученные при производстве осмотра с применением фото- и киносъемки, иных установленных способов фиксации вещественных доказательств, прилагаются к соответствующему протоколу.9. Протокол осмотра места совершения административного правонарушения подписывается составившим его должностным лицом, а также лицами, участвовавшими в производстве осмотра. Копии протокола осмотра места совершения администрати

In [68]:
deepseek_dict = {}
# for i in tqdm(range(1)):
for i in tqdm(range(len(contexts))):
    deepseek_dict[i] = {}
    n = i
    instruction = contexts['instruction'][n]
    question = contexts['input'][n]
    gold_answer = contexts['output'][n]
    for emb in ["labse", "mini", "multi"]:
        deepseek_dict[i][emb] = {}
        # for ranger in ["faiss", "bm25", "ensemble", "custom"]:
        for ranger in ["custom"]:
            column = "_".join([emb, ranger, "top"])
            list_contexts = [contexts[f'{column}{i}'][n] for i in range(5)]
            print(list_contexts)

100%|██████████| 120/120 [00:00<00:00, 4890.75it/s]

['Глава 6. Доказательства и доказывание Статья 61. Допустимость доказательств 1. Доказательства являются допустимыми, если они отвечают требованиям, указанным в статье 59 настоящего Кодекса. Обстоятельства административного дела, которые согласно закону должны быть подтверждены определенными средствами доказывания, не могут подтверждаться никакими иными доказательствами.2. Суд признает доказательства недопустимыми по письменному ходатайству лица, участвующего в деле, или по собственной инициативе.', 'Глава 6. Доказательства и доказывание Статья 60. Допустимость доказательств Обстоятельства дела, которые в соответствии с законом должны быть подтверждены определенными средствами доказывания, не могут подтверждаться никакими другими доказательствами.', 'их доказательств перед другими и определять, какое судебное постановление должно быть принято при новом рассмотрении дела. Дополнительные доказательства судом кассационной инстанции не принимаются.4. Указания вышестоящего суда о толковании

In [ ]:
# Цикл обработки всего датасета
# for i in tqdm(range(len(contexts))):
deepseek_dict = {}
# for i in tqdm(range(1)):
for i in tqdm(range(len(contexts))):
    deepseek_dict[i] = {}
    n = i
    instruction = contexts['instruction'][n]
    question = contexts['input'][n]
    gold_answer = contexts['output'][n]
    for emb in ["labse", "mini", "multi"]:
        deepseek_dict[n][emb] = {}
        # for ranger in ["faiss", "bm25", "ensemble", "custom"]:
        for ranger in ["custom"]:
            column = "_".join([emb, ranger, "top"])
            list_contexts = [contexts[f'{column}{j}'][n] for j in range(5)]
    
            prompt = """
            
            Тебе будет предоставлен контекст и вопрос по нему на юридическую тематику,
            и набор контекстов для ответа на поставленный вопрос, которые надо оценить относительно эталонного ответа на этот вопрос. 
            
            Твоя задача - оценить представленные контексты по трём критериям:

            1)Релевантность контекста (1-5) - насколько каждый из представленных контекстов необходим для ответа на поставленный вопрос.
            2)Полнота контекста (1-5) - насколько хорошо контекст обогащен необходимой для ответа информацией.
            3)Согласованность контекстов (1-5) - насколько хорошо все представленные контексты согласуются друг с другом.
            4)Общее покрытие (1-5) - можно ли точно ответить на вопрос, исходя только из знаний представленных в контекстах.
            5)Оценка, с помощью эталонного ответа (1-5) - представь, что тебе необходимо ответить на поставленный вопрос с учетом представленных
            контекстов, оцени свой предполагаемый ответ относительно эталонного ответа, который был тебе представлен, согласуется ли твой предполагаемый ответ
            с эталонным в фактах и выводах.
            6)Корректность трактовки законов (1-5) - можно ли с помощью представленных контекстов ссылаться на существующие российские законы.
            
            Прежде чем поставить оценку, проведи анализ по соответсвующему критерию.
            
            Пожалуйста, убедись, что ты внимательно прочитал и понял эти инструкции. 
            Пожалуйста, держи эти инструкции открытыми во время просмотра и обращайся к ним по мере необходимости.

            Пример исходных данных:
            Контекст: Ты - юридический консультант. Тебе даётся следующий контекст: Врач-невролог ФИО1 был привлечен к административной ответственности по ч. 1 ст. 5.61 КоАП РФ за оскорбление коллеги в рабочем помещении. Мировой судья признал его виновным как должностное лицо и назначил штраф в размере 30 000 рублей. ФИО1 подал жалобу, оспаривая квалификацию его как должностного лица, поскольку он не обладает организационно-распорядительными или административно-хозяйственными функциями, а дело было возбуждено в отношении него как физического лица. Исходя из этого, ответь на вопрос: 
            Вопрос: Могут ли привлечь меня к ответственности как должностное лицо, если я обычный врач без управленческих функций?
            Набор контекстов:
            1)м, в случае, если судья признает обязательным присутствие в судебном заседании указанных лиц.3.10. Неявка в судебное заседание надлежащим образом извещенного уполномоченного должностного лица органа внутренних дел или органа исполнительной власти в области обеспечения безопасности, подавшего ходатайство о продлении срока содержания иностранного гражданина или лица без гражданства в специальном учреждении, не препятствует рассмотрению и разрешению такого ходатайства при наличии документов, подтве
            2)ение вопросов, не входящих в компетенцию ее высшего органа и созданного в соответствии с пунктом 4 настоящей статьи коллегиального органа управления.4. Наряду с исполнительными органами, указанными в пункте 3 настоящей статьи, в корпорации может быть образован в случаях, предусмотренных настоящим Кодексом, другим законом или уставом корпорации, коллегиальный орган управления (наблюдательный или иной совет), контролирующий деятельность исполнительных органов корпорации и выполняющий иные функции,
            3)й Федерации;отсутствие соответствующего документа об образовании и (или) о квалификации, если выполнение работы требует специальных знаний в соответствии с федеральным законом или иным нормативным правовым актом;заключение трудового договора в нарушение постановления судьи, органа, должностного лица, уполномоченных рассматривать дела об административных правонарушениях, о дисквалификации или ином административном наказании, исключающем возможность исполнения работником обязанностей по трудовому
            4)ее двух рабочих дней, следующих за днем выявления таких фактов, в письменной форме уведомить об этом Банк России (с указанием соответствующих фактов);2) не позднее одного месяца со дня выявления таких фактов в порядке, предусмотренном трудовым законодательством, освободить указанное лицо от занимаемой должности (освободить от временного исполнения обязанностей по этой должности);3) уведомить Банк России об освобождении указанного лица от занимаемой должности (освобождении от временного исполнени
            5)ющее функции (лица, осуществляющие функции) единоличного исполнительного органа указанных юридических лиц, должны соответствовать следующим требованиям:отсутствие неснятой или непогашенной судимости за совершение преступления;отсутствие факта применения в отношении лица административного наказания в виде дисквалификации в соответствии с Кодексом Российской Федерации об административных правонарушениях (законодательным актом иностранного государства, если им предусмотрено такое наказание), если н
            Эталонный ответ: Нет, если вы не выполняете организационно-распорядительные или административно-хозяйственные функции (примечание к ст. 2.4 КоАП РФ). Для привлечения как должностного лица необходимо, чтобы правонарушение было связано с исполнением служебных обязанностей, чего в вашем случае нет.

            Формат ответа:
            Рассуждения и обоснования ответов (здесь можешь описать все размышления)
            Далее ответ в представленном формате:
            Оценка:
            1)Релевантность контекста (1-5): (число).
            2)Полнота контекста (1-5): (число).
            3)Согласованность контекстов (1-5): (число).
            4)Общее покрытие (1-5): (число).
            5)Оценка, с помощью эталонного ответа (1-5): (число).
            6)Корректность трактовки законов (1-5): (число).
            Итоговый балл (среднее по всем критериям) без округления:

            Входные данные
            Контекст: {}
            Вопрос: {}
            Набор контекстов: 
            1){}
            2){}
            3){}
            4){}
            5){} 
            Эталонный ответ: {}       
            
            Выведи итоговый балл (среднее по всем критериям) без округления.
            """



            prompt = prompt.format(instruction, question, *list_contexts, gold_answer)
            
            api_key = "********************************"  # мой токен deepseek


            # deepseek иногда отъебывает, я даю ему 3 попытки иначе скип итерация
            for i in range(3):
                try:
                    
                    # # бэкап в файл 
                    # with open('base_geval_backup.json', 'r', encoding='utf-8') as f:
                    #     base_geval_res = json.load(f)

                    # вызов deepseek 
                    otvet = deepseek_api_call(prompt, api_key)
                    deepseek_dict[n][emb][ranger] = otvet
                    # base_geval_res.append(otvet)
                    
                    # print(otvet)
                    
                    with open('deepseek_answers.json', 'w', encoding='utf-8') as f:
                        json.dump(deepseek_dict, f, ensure_ascii=False)
                    
                    break
            
                except KeyError as e:
                    print(f'Error {e}')


100%|██████████| 120/120 [3:05:33<00:00, 92.78s/it] 


In [71]:
print(deepseek_dict.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119])


In [72]:
for key, val in deepseek_dict.items():
    for emb, res in val.items():
        print(emb)

labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
multi
labse
mini
mul

In [49]:
print(otvet)

Рассуждения и обоснования ответов:

1) **Релевантность контекста (1-5)**:  
   - Контексты 1 и 2 касаются допустимости доказательств, что частично связано с вопросом о фотофиксации.  
   - Контексты 3, 4 и 5 не имеют прямого отношения к обязательности фотофиксации.  
   - Оценка: 2 (только 2 контекста частично релевантны).

2) **Полнота контекста (1-5)**:  
   - Контексты 1 и 2 содержат общие положения о доказательствах, но не раскрывают конкретно вопрос о фотофиксации.  
   - Остальные контексты не добавляют полезной информации.  
   - Оценка: 2 (информация недостаточно полная для ответа на вопрос).

3) **Согласованность контекстов (1-5)**:  
   - Контексты 1 и 2 согласуются между собой, но остальные не связаны с ними или друг с другом.  
   - Оценка: 2 (согласованность низкая, так как большинство контекстов не относятся к теме).

4) **Общее покрытие (1-5)**:  
   - Из представленных контекстов невозможно точно ответить на вопрос, так как отсутствует информация о фотофиксации как обяз

In [50]:
list_contexts

['Глава 6. Доказательства и доказывание Статья 61. Допустимость доказательств 1. Доказательства являются допустимыми, если они отвечают требованиям, указанным в статье 59 настоящего Кодекса. Обстоятельства административного дела, которые согласно закону должны быть подтверждены определенными средствами доказывания, не могут подтверждаться никакими иными доказательствами.2. Суд признает доказательства недопустимыми по письменному ходатайству лица, участвующего в деле, или по собственной инициативе.',
 'Глава 6. Доказательства и доказывание Статья 60. Допустимость доказательств Обстоятельства дела, которые в соответствии с законом должны быть подтверждены определенными средствами доказывания, не могут подтверждаться никакими другими доказательствами.',
 'их доказательств перед другими и определять, какое судебное постановление должно быть принято при новом рассмотрении дела. Дополнительные доказательства судом кассационной инстанции не принимаются.4. Указания вышестоящего суда о толкован

In [52]:
gold_answer

'Нет, фотофиксация не является обязательной по КоАП РФ. Согласно ст. 26.11 КоАП РФ, доказательства оцениваются в совокупности. Если другие материалы (схема, объяснения) достаточны для установления обстоятельств, отсутствие фото не повлияет на законность постановления.'

In [76]:
import re

text = """
Рассуждения и обоснования ответов:

1) **Релевантность контекста (1-5)**:  
   - Контексты 1 и 2 касаются допустимости доказательств, что частично связано с вопросом о фотофиксации.  
   - Контексты 3, 4 и 5 не имеют прямого отношения к обязательности фотофиксации.  
   - Оценка: 2 (только 2 контекста частично релевантны).

2) **Полнота контекста (1-5)**:  
   - Контексты 1 и 2 содержат общие положения о доказательствах, но не раскрывают конкретно вопрос о фотофиксации.  
   - Остальные контексты не добавляют полезной информации.  
   - Оценка: 2 (информация недостаточно полная для ответа на вопрос).

3) **Согласованность контекстов (1-5)**:  
   - Контексты 1 и 2 согласуются между собой, но остальные не связаны с ними или друг с другом.  
   - Оценка: 2 (согласованность низкая, так как большинство контекстов не относятся к теме).

4) **Общее покрытие (1-5)**:  
   - Из представленных контекстов невозможно точно ответить на вопрос, так как отсутствует информация о фотофиксации как обязательном действии.  
   - Оценка: 1 (информации недостаточно для ответа).

5) **Оценка, с помощью эталонного ответа (1-5)**:  
   - Эталонный ответ указывает, что фотофиксация не обязательна, но в контекстах нет прямого подтверждения этого.  
   - Оценка: 2 (предполагаемый ответ не будет полностью соответствовать эталонному из-за недостатка информации).

6) **Корректность трактовки законов (1-5)**:  
   - Контексты 1 и 2 корректно цитируют КоАП РФ, но не дают полного ответа на вопрос.  
   - Оценка: 3 (законы цитируются верно, но не в полном объеме).

**Итоговый балл (среднее по всем критериям) без округления**:  
(2 + 2 + 2 + 1 + 2 + 3) / 6 = 2.0  

Оценка:  
1) Релевантность контекста (1-5): 2.  
2) Полнота контекста (1-5): 2.  
3) Согласованность контекстов (1-5): 2.  
4) Общее покрытие (1-5): 1.  
5) Оценка, с помощью эталонного ответа (1-5): 2.  
6) Корректность трактовки законов (1-5): 3.  
Итоговый балл (среднее по всем критериям) без округления: 2.0
"""

# Ищем все числовые оценки в тексте
matches = re.findall(
    r'(?:Оценка|балл).*?([0-9]+(?:\.[0-9]+)?)', 
    text, 
    flags=re.IGNORECASE
)

# Преобразуем найденные значения в числа
scores = [float(match) if '.' in match else int(match) for match in matches]

print(scores)

[2, 2, 2, 1, 1, 2, 3, 1, 2.0]


In [85]:
deepseek_dict.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119])

In [87]:
pd.DataFrame.from_dict(deepseek_dict).transpose()

,labse,mini,multi
0,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...
1,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...
2,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...
3,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...
4,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...
...,...,...,...
115,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...
116,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...
117,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...
118,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...,{'custom': 'Рассуждения и обоснования ответов:...


In [290]:
import re
import json

deepseek_answers_df = {}

with open("deepseek_answers.json", "r") as f:
    deepseek_dict = json.load(f)

for n in range(len(deepseek_dict)):
    deepseek_answers_df[n] = {}
    
    for method, val in deepseek_dict[str(n)].items():
        text = val['custom']
        scores = []

        try:
            main_scores = []
            final_score = None
            grades_block = ""

            # Part 1: Первым приоритетом проверяем формулу
            formula_match = re.search(r'\(([0-9+ .]+)\)\s*/\s*\d+\s*=\s*([0-9.]+)', text)
            if formula_match:
                try:
                    # Парсим числа из формулы
                    numbers = [float(x.strip()) for x in formula_match.group(1).split('+')]
                    if len(numbers) >= 6:
                        main_scores = numbers[:6]
                        # Парсим итоговый балл из формулы
                        final_score = float(formula_match.group(2).strip())
                except:
                    pass

            # Part 2: Если не нашли в формуле, ищем стандартный формат
            if not main_scores:
                start_idx = text.find("Оценка:")
                if start_idx != -1:
                    specific_start = text.find("1) Релевантность контекста", start_idx)
                    end_idx = text.find("Итоговый балл", specific_start)
                    grades_block = text[specific_start:end_idx]

                    for line in grades_block.split('\n'):
                        if re.match(r'\s*\d+\)', line):
                            match = re.search(r':\s*([0-9.]+)', line)
                            if match and len(main_scores) < 6:
                                value = match.group(1).rstrip('.').strip()
                                if value.replace('.', '', 1).isdigit():
                                    num = float(value) if '.' in value else int(value)
                                    main_scores.append(num)

            # Part 3: Извлечение итогового балла (приоритет формуле)
            if not final_score:
                final_match = re.search(
                    r'Итоговый балл.*?[=:]\s*([0-9.]+)', 
                    text, 
                    re.DOTALL | re.IGNORECASE
                )
                if final_match:
                    final_value = final_match.group(1).rstrip('.').strip()
                    if final_value.replace('.', '', 1).isdigit():
                        final_score = float(final_value)

            # Формируем итоговый список
            scores = main_scores[:6] + [final_score]
            scores = [x if x is not None else None for x in scores]
            scores = scores[:7] if len(scores) >=7 else scores + [None]*(7-len(scores))

        except Exception as e:
            print(f"Ошибка обработки {method}: {str(e)}")
            scores = [None] * 7

        deepseek_answers_df[n][method] = scores

In [ ]:
def extract_final_grades(text: str) -> list:
    """
    Извлекает 6 числовых значений из блока с итоговым баллом.
    Возвращает список из 6 чисел типа float.
    """
   
    pattern = r"Итоговый балл.*?\(([\d. +]+)\)"
    match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
    
    if not match:
        return []
    
    numbers_str = match.group(1)
    

    numbers = []
    for num in numbers_str.split('+'):

        cleaned_num = num.strip()
        if cleaned_num:
            try:
                numbers.append(float(cleaned_num))
            except ValueError:
                continue
    
    return numbers[:6]

for n in [38, 39, 42, 50, 51, 51, 51, 52, 53, 55, 63, 63, 72, 74, 81, 82, 82, 93, 95, 96, 98, 98, 99, 100, 101, 109, 115]:
    # deepseek_answers_df[n] = {}
    
    for method, val in deepseek_dict[str(n)].items():
        if None in deepseek_answers_df[n][method][:-1]:
            text = val['custom']
            scores = extract_final_grades(text)
            deepseek_answers_df[n][method] = scores


In [297]:
error

[(76, 'labse', [2.0, None, None, None, None, None, None]),
 (76, 'mini', [2.0, None, None, None, None, None, None]),
 (108, 'labse', [2.6, None, None, None, None, None, None]),
 (112, 'mini', [2.0, None, None, None, None, None, None])]

In [296]:
error = []
for key, val in deepseek_answers_df.items():
    for meth, numb in val.items():
        try:
            if None in numb:
                # print(numb)
                if None in numb[:-1]:
                    error.append((key, meth, numb))
                    
            
            # elif abs(np.mean(numb[:-1]) - numb[-1]) > 0.1:
                

        except: 
            print(key, 'err')
            error.append((key, meth, numb))

In [302]:
methods = {}
methods['mini'] = []
methods['multi'] = []
methods['labse'] = []
for key, val in deepseek_answers_df.items():
    for meth, numb in val.items():
        try:
            if len(numb) > 1:
                methods[meth].append(float(np.mean(numb[:-1])))
            else:
                methods[meth].append(numb[0])
        except Exception as e:
            print(e)
            print(numb)
            print(key, meth)


In [307]:
print(f"""
    Mini: {np.mean(methods['mini'])} \n
    Multi: {np.mean(methods['multi'])} \n
    Labse: {np.mean(methods['labse'])} 
""")


    Mini: 1.9194138888888892 

    Multi: 2.409916666666667 

    Labse: 2.4302777777777775 



In [298]:
deepseek_answers_df[50]['mini'] = [1.03]
deepseek_answers_df[51]['mini'] = [2.06]
deepseek_answers_df[93]['multi'] = [3.5]
deepseek_answers_df[96]['labse'] = [1.0]
deepseek_answers_df[101]['multi'] = [1.0]
deepseek_answers_df[76]['mini'] = [2.26]
deepseek_answers_df[76]['labse'] = [3.0]
deepseek_answers_df[108]['labse'] = [3.53]
deepseek_answers_df[112]['mini'] = [2.633]

In [312]:
print(deepseek_dict['112']['labse']['custom'])

Рассуждения и обоснования ответов:

1) **Релевантность контекста (1-5)**:  
   - Контекст 2 частично релевантен, так как касается возмещения расходов, но не конкретно юридических услуг.  
   - Контекст 4 касается допустимости доказательств, что может быть косвенно связано с подтверждением расходов на юриста.  
   - Остальные контексты (1, 3, 5) не имеют отношения к вопросу.  
   Оценка: 2 (только частичная релевантность).

2) **Полнота контекста (1-5)**:  
   - Контексты не содержат прямых указаний на взыскание расходов на юриста или нотариуса.  
   - Нет ссылок на ст. 100 ГПК РФ или аналогичные нормы.  
   Оценка: 1 (информация недостаточна для ответа).

3) **Согласованность контекстов (1-5)**:  
   - Контексты не связаны между собой и не образуют единой логической цепочки.  
   Оценка: 1 (отсутствие согласованности).

4) **Общее покрытие (1-5)**:  
   - Невозможно точно ответить на вопрос, так как отсутствуют ключевые нормы (например, ст. 100 ГПК РФ).  
   Оценка: 1 (недостаточное по